<a href="https://colab.research.google.com/github/agusjanardana/Tubes_ML2021/blob/main/k-means/k_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Inisialisasi kebutuhan library
# https://www.kaggle.com/prashant111/knn-classifier-tutorial
import pandas as pd
import io
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files